pip install --upgrade pip
sudo apt-get update
sudo apt-get install -y python3-opencv
pip install tensorflow opencv-python seaborn pandas scikit-learn matplotlib numpy

In [ ]:
import os
import sys
sys.path.append("../../..")
# openCV
import cv2
# numpy
import numpy as np
# matplotlib
import matplotlib.pyplot as plt
# class util functions
from utils.imutils import jimshow as show
from utils.imutils import jimshow_channel as show_channel
import utils.classifier_utils as clf_util

# Import sklearn metrics
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# cifar10 dataset
from tensorflow.keras.datasets import cifar10

In [ ]:
# loading the cifar10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
X_train_list=[]
X_test_list=[]

for image in X_test: # preprocessing the testing images
    gray_test = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_test/255 # normalize
    X_test_list.append(normalized) # add to list
X_preprocess_test_array = np.array(X_test_list) # turn list back into an array
# reshape
X_preprocess_test = X_preprocess_test_array.reshape(-1, 1024)

for image in X_train: # preprocessing the training images
    gray_train = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_train/255 # normalize
    X_train_list.append(normalized) # add to list
X_preprocess_train_array = np.array(X_train_list) # turn list back into an array
# reshape
X_preprocess_train = X_preprocess_train_array.reshape(-1, 1024)

X_preprocess_train.shape
X_preprocess_test.shape

In [ ]:
# View an image
# plt.imshow(X_preprocess_train[0])
# plt.show()

In [ ]:
# making a logistic regression classifier
clf = LogisticRegression(tol=0.1, 
                         solver='saga',
                         multi_class='multinomial').fit(X_preprocess_train, y_train)

In [ ]:
# defining labels
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# testing accuracy and making a classification report for the logistic regression classifier
y_pred = clf.predict(X_preprocess_test)
cm = metrics.classification_report(y_test, y_pred, target_names=classes)
print(cm)

# saving classification report as a .txt file
text_file = open(r'../output/LR_classification_report.txt', 'w')
text_file.write(cm)
text_file.close()

In [ ]:
sliceddataX = X_preprocess_train[0:1000]
sliceddataXtest = X_preprocess_test[0:1000]
sliceddataY = y_train[0:1000]
sliceddataYtest = y_test[0:1000]

In [ ]:
# neural network classifier

#create the classifier
classifier = MLPClassifier(activation = "logistic",
                            hidden_layer_sizes = (2,), # 20 neurons in our hidden layer
                            max_iter=500, # 1000 epochs
                            random_state = 42) # random state for reproducibility

# fit the classifier to the data
classifier.fit(sliceddataX, sliceddataY)

In [ ]:
# get predictions
y_pred = classifier.predict(sliceddataXtest)

# use confusion matrix to check performance
#metrics.ConfusionMatrixDisplay.from_estimator(classifier,           # the classifier name
                                            #sliceddataX,            # the training features
                                            #sliceddataY,            # the training labels
                                            #cmap=plt.cm.Blues,      # make the colours prettier
                                            #labels=["FAKE", "REAL"])# the labels in the data arranged alphabetically

# get classification report
classifier_metrics = metrics.classification_report(sliceddataYtest, y_pred, target_names = classes)
print(classifier_metrics)

# saving classification report as a .txt file
text_file = open(r'../output/MLP_classification_report.txt', 'w')
text_file.write(classifier_metrics)
text_file.close()

# plotting loss curves. should ideally have a smooth, steep downwards slope which ends in a plateau
plt.plot(classifier.loss_curve_)
plt.title("Loss curve during training", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Loss score')
# save output
plt.savefig('../output/loss_curve.png')
plt.show()

In [1]:
from logistic_regression import logisticregression
logisticregression()

2024-03-12 15:43:15.840573: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 15:43:15.844458: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 15:43:15.894714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 15:43:16.781864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=Tr

              precision    recall  f1-score   support

    airplane       0.34      0.38      0.36      1000
  automobile       0.36      0.39      0.37      1000
        bird       0.27      0.20      0.23      1000
         cat       0.22      0.15      0.18      1000
        deer       0.25      0.22      0.24      1000
         dog       0.29      0.31      0.30      1000
        frog       0.28      0.33      0.30      1000
       horse       0.31      0.31      0.31      1000
        ship       0.34      0.40      0.37      1000
       truck       0.39      0.44      0.41      1000

    accuracy                           0.31     10000
   macro avg       0.31      0.31      0.31     10000
weighted avg       0.31      0.31      0.31     10000

